# Investigation of Distributed CNN Training Performances within the context of a Sea Turtle Face Classifier

## CNN Distribution

In [24]:
#run this cell to release the ram that the workers are hoarding. 
#only to be used for debugging memory leeks, there is a fancy way for automatic trimming
import ctypes

def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

client.run(trim_memory)

{'tcp://172.19.0.3:42719': 1,
 'tcp://172.19.0.4:42091': 1,
 'tcp://172.19.0.5:39753': 1,
 'tcp://172.19.0.7:37421': 1,
 'tcp://172.19.0.8:39819': 1,
 'tcp://172.19.0.9:38657': 1}

In [2]:
# are you on cloud veneto?
onCV = False
if not onCV: 
    !pip install scikit-image

     |████████████████████████████████| 14.0 MB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 68.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 92.5 MB/s eta 0:00:01
     |████████████████████████████████| 195 kB 95.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 85.0 MB/s eta 0:00:01


In [6]:
import matplotlib.pyplot as plt
import numpy as np
import dask.dataframe as dd
import dask.array as da
from pathlib import Path
from dask.distributed import wait
from dask.distributed import Client
import pickle

#from dask_image.imread import imread
from skimage import io, transform
import dask.array as da
import dask.delayed

from tensorflow.keras import models, layers
import tensorflow as tf

import pandas as pd

import re
import os

import time
import timeit

In [7]:
#This will stop the dask worker and scheduler processes on the respective machines
# client.shutdown()

In [5]:
from dask.distributed import Client
if onCV:
    # must run to following command on this jupyter machine to set up cluster.  
    # dask-ssh 10.67.22.{178,193,212,246,25,52} --scheduler 10.67.22.233 --log-directory /home/ubuntu/daskShared_local/logs --ssh-username ubuntu --ssh-private-key /home/ubuntu/.ssh/dasktest
    client = Client('10.67.22.233:8786')
    print('Connecting to CV cluster')

else:
    #after doing docker-compose up the cluster is ready to connect
    client = Client('dask-scheduler:8786')
    print('Connecting to docker cluster')
    #dask-scheduler is an alias for the ip address of the scheduler. I believe it is set up in the image of the jupyter notebook machine. 

client

Connecting to docker cluster


<Client: 'tcp://172.19.0.2:8786' processes=6 threads=6, memory=11.62 GiB>

In [ ]:
# # This cell is used to run commands on all of the workers with parameko ssh
# import paramiko

# scheduler_ip = '10.67.22.233'
# workers_ip = ['10.67.22.52','10.67.22.178','10.67.22.25','10.67.22.246','10.67.22.193','10.67.22.212']
# all_ip = ['10.67.22.233','10.67.22.52','10.67.22.178','10.67.22.25','10.67.22.246','10.67.22.193','10.67.22.212']


# def ssh_cmd(hostname, cmd_list):
#     k = paramiko.RSAKey.from_private_key_file("/home/ubuntu/.ssh/dasktest")
#     c = paramiko.SSHClient()
#     c.load_host_keys(filename="/home/ubuntu/.ssh/known_hosts")
#     c.set_missing_host_key_policy(paramiko.AutoAddPolicy())
#     c.connect(hostname = hostname, username = "ubuntu", pkey = k)
#     for command in cmd_list:
#         print ("Executing {}".format( command ))
#         stdin , stdout, stderr = c.exec_command(command)
#         print(stdout.read())
#         print( "Errors")
#         print(stderr.read())
#         c.close()


# # #use this cell to run a cmd command on all the workers
# # # #set up worker
# # # # workers_ip = ['10.67.22.40','10.67.22.195']
# # mkdir = ['mkdir /home/ubuntu/daskShared_local/face']
# # sshfs = ["sudo sshfs -o allow_other,default_permissions,IdentityFile=/home/ubuntu/.ssh/dasktest,StrictHostKeyChecking=accept-new ubuntu@10.67.22.18:/home/ubuntu/daskShared_remote /home/ubuntu/daskShared_local"]
# # touch = ['touch /home/ubuntu/daskNotShared/progress/train.pr']
# # chmod = ['sudo chmod 666 /home/ubuntu/daskNotShared/daniel_logs/general.txt']
# reboot = ['sudo reboot']
# for mac in all_ip:
#     ssh_cmd(mac,reboot)


In [24]:
#run to check that the sheduler and workers have the same enviroments
#client.get_versions(check=True)

In [25]:
#run to delete everthing on the workers
#client.restart()

In [26]:
#run to see what is stored on each worker
#client.has_what()

In [11]:
#set paths
if onCV: path_lbl = Path('/home','ubuntu','daskShared_local','labels.csv')
else: path_lbl = Path('/opt','workspace','data','labels.csv')

if onCV: path_img = Path('/home','ubuntu','daskShared_local','images')
else: path_img = Path('/opt','workspace','data','images')

#this file will be writted to all of the workers into this path.
#this path should not be in the folder that is shared between workers.
if onCV: path_weights = Path('/home','ubuntu','daskNotShared','weights.pkl')
else: path_weights = Path('/weights/weights.pkl')
    
if onCV: path_face = Path('/home','ubuntu','daskShared_local','face')
else: path_face = Path('/opt','workspace','data','face')

if onCV: path_noface = Path('/home','ubuntu','daskShared_local','noface')
else: path_noface = Path('/opt','workspace','data','noface')


In [12]:
#make the g6.py file to move some functions straight to the workers
#This reduces traffic when running these functions on the workers
# It was also the only way to get the cnn on the worker, as it couldn't be passed in a function.

#remove old g6.py file
try:
    os.system('rm g6.py')
    client.run(lambda dask_worker: os.remove(os.path.join(dask_worker.local_directory, "g6.py")))
except: print("No g6.py file was on the workers or the current machine, adding it now")
#the code below is displayed as such for syntax highlighting. It is then made into a string and written to a file to be sent to the workers and used as a g6 package
from tensorflow.keras import models, layers
import tensorflow as tf
import pickle
def build_cnn():
    cnn = models.Sequential([
        layers.Conv2D(filters=30, kernel_size=(3,3), activation='relu', input_shape=(198,198,3)),
        layers.MaxPooling2D((10,10)),
        layers.Conv2D(filters=30, kernel_size=(3,3), activation='relu', input_shape=(198,198,3)),
        layers.MaxPooling2D((10,10)),
        #dense
        layers.Flatten(),
        layers.Dense(50, activation='relu'),
        layers.Dense(2, activation='softmax')
    ])
    # compile
    cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    #summary
    #print(cnn.summary())
    return cnn

cnn = build_cnn()

#extracting the weights from a cnn
def get_w(cnn):
    layer_w = [lay.get_weights() for lay in cnn.layers]
    return layer_w

#changing the weights of the cnn
def set_w(cnn,weights):
    for lay,w in zip(cnn.layers, weights):
        lay.set_weights(w)
    return cnn


#make a pickle file on the worker with the initial weights
def weights_init():
    weights = get_w(cnn)
    f_weights = open(Path(path_weights), 'wb')
    pickle.dump(weights, f_weights)                     
    f_weights.close()
    
# once the weights have been trained. This brings them from the workers to this machine for validation
def weights_get():
    f_weights = open(path_weights,'rb')
    weights = pickle.load(f_weights)
    f_weights.close()
    return weights
#weights = list(client.run(weights_get).values())

#now we have the best weights we need to write them to each worker so they can re-train from the best weights
def weights_set_best(weights_best):
    f_weights = open(path_weights,'wb')
    pickle.dump(weights_best,f_weights)
    f_weights.close()
    del f_weights, weights_best
    
contents = "\
from tensorflow.keras import models, layers\n\
import tensorflow as tf\n\
import pickle\n\
path_weights='"+str(path_weights)+"'\n\
def build_cnn():\n\
    cnn = models.Sequential([\n\
        layers.Conv2D(filters=30, kernel_size=(3,3), activation='relu', input_shape=(198,198,3)),\n\
        layers.MaxPooling2D((10,10)),\n\
        layers.Conv2D(filters=30, kernel_size=(3,3), activation='relu', input_shape=(198,198,3)),\n\
        layers.MaxPooling2D((10,10)),\n\
        #dense\n\
        layers.Flatten(),\n\
        layers.Dense(50, activation='relu'),\n\
        layers.Dense(2, activation='softmax')\n\
    ])\n\
    # compile\n\
    cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])\n\
    #summary\n\
    #print(cnn.summary())\n\
    return cnn\n\
\n\
cnn = build_cnn()\n\
\n\
def get_w(cnn):\n\
    layer_w = [lay.get_weights() for lay in cnn.layers]\n\
    return layer_w\n\
#changing the weights of the cnn\n\
def set_w(cnn,weights):\n\
    for lay,w in zip(cnn.layers, weights):\n\
        lay.set_weights(w)\n\
    return cnn\n\
\n\
#make a pickle file on the worker with the initial weights\n\
def weights_init():\n\
    weights = get_w(cnn)\n\
    f_weights = open(path_weights, 'wb')\n\
    pickle.dump(weights, f_weights)\n\
    f_weights.close()\n\
    \n\
# once the weights have been trained. This brings them from the workers to this machine for validation\n\
def weights_get():\n\
    f_weights = open(path_weights,'rb')\n\
    weights = pickle.load(f_weights)\n\
    f_weights.close()\n\
    return weights\n\
#now we have the best weights we need to write them to each worker so they can re-train from the best weights\n\
def weights_set_best(weights_best):\n\
    f_weights = open(path_weights,'wb')\n\
    pickle.dump(weights_best,f_weights)\n\
    f_weights.close()\n\
    del f_weights, weights_best"

#write the file this this machine on the same dir as this notebook
f_py = open("g6.py",'w')
f_py.write(contents)
f_py.close()

#upload the file to the workers so it can be imported on the workers
client.upload_file('g6.py')

#use the following commmand to remove unwanted files that have been uploaded
#client.run(lambda dask_worker: os.remove(os.path.join(dask_worker.local_directory, "file.py")))

rm: cannot remove 'g6.py': No such file or directory


No g6.py file was on the workers or the current machine, adding it now


{'tcp://172.19.0.3:42719': {'status': 'OK'},
 'tcp://172.19.0.4:42091': {'status': 'OK'},
 'tcp://172.19.0.5:39753': {'status': 'OK'},
 'tcp://172.19.0.7:37421': {'status': 'OK'},
 'tcp://172.19.0.8:39819': {'status': 'OK'},
 'tcp://172.19.0.9:38657': {'status': 'OK'}}

In [13]:
#You can checked it worked by initalising the weights
# import g6
# client.run(g6.weights_init)

In [14]:
#define the function that will train on each partition. It will load weights, train, then write the improved weights over the same file
@dask.delayed
def train(chunk,monitor=False):
    if monitor:
        def log(log="",clear=False):
            f_log = open("/home/ubuntu/daskNotShared/daniel_logs/general.txt",'a')
            f_log.write(log+'\n')
            f_log.close()

        f_progress = open("/home/ubuntu/daskNotShared/progress/train.pr",'a')
        f_progress.write('1 ')
        f_progress.close()

        log('making partition data cnn ready')
        
    #convert the chunk to a form that will go into the cnn
    face = chunk[:,0,:,:,:]
    noface = chunk[:,1,:,:,:]
    labelsF = np.ones(len(face))
    labelsNF = np.zeros(len(noface))
    
    data = tf.convert_to_tensor(np.concatenate((face,noface), axis = 0))
    labels = tf.convert_to_tensor(np.concatenate((labelsF,labelsNF)))
    del face, labelsF, noface, labelsNF
    #---------------------------------------------------
    if monitor: log('getting weights from file')
    #get init/best weights from pickle file 
    f_weights = open(path_weights,'rb')
    weights = pickle.load(f_weights)
    f_weights.close()
    del f_weights
    if monitor: log('training on partition data')
    cnn = g6.set_w(g6.cnn, weights)
    fit = cnn.fit(data,labels, epochs = 10, shuffle=True)
    if monitor: log('getting and saving weights\n---------------------------------------\n\n')
    weights = g6.get_w(cnn)
    f_weights = open(path_weights, 'wb')
    pickle.dump(weights, f_weights)                     
    f_weights.close()
    del f_weights, weights, fit, data, labels, cnn

#because this function returnes nothing, delayed was more convenient than dask.array.map_blocks()

In [15]:
#once the weights have been trained, we need to evaluate the accuracy they provide
# The workers find the acc of different chunks in parralell. The mean acc of the chunks is the overall accuracy.
#The function loads the weights from the same place the trainer saved them.
@dask.delayed
def evaluate(chunk,weights):
    #convert the chunk to a form that will go into the cnn
    face = chunk[:,0,:,:,:]
    noface = chunk[:,1,:,:,:]
    labelsF = np.ones(len(face))
    labelsNF = np.zeros(len(noface))
    
    data = tf.convert_to_tensor(np.concatenate((face,noface), axis = 0))
    labels = tf.convert_to_tensor(np.concatenate((labelsF,labelsNF)))
    del face, labelsF, noface, labelsNF
    
    cnn = g6.set_w(g6.cnn,weights)
    loss, acc = cnn.evaluate(data,labels)
    del cnn, loss
    return acc


In [16]:
from dask.array.image import imread
#this will load the saved images from file into a dask array
# it is stacked so the face images are on top of the noface images, this is the second axis
def getData():    
    face = imread(str(path_face)+'/*')
    noface = imread(str(path_noface)+'/*')
    #display(face)
    #display(noface)
    data = [face,noface]
    data = da.stack(data,axis=1)
    data = data.rechunk(chunks=(1,2,198,198,3))
    #display(data)
    del face, noface
    return data

In [17]:
#This cell is used for testing of the training and evaluate functions
# import g6
# data = getData()
# data = data[0:20].rechunk(chunks=(10,2,198,198,3)).to_delayed().ravel()
# display(data)

#test = data.rechunk(chunks=(20,2,198,198,3))
#test = test.to_delayed().ravel()
#train_go = [train(te) for te in test]
# dd.compute(*train_go)

#display(test)
#result = da.map_blocks(train,test,meta=np.array((), dtype=np.bool_),chunks=(1999,1))
#result = result.persist()
#display(result)
# weights = list(client.run(g6.weights_get).values())
# acc = []
# for j in range(len(weights)):
#     print('evaluation at itteration:',1,'of weights',j)
#     #distrbuted evaluation of weights, returns accuracies of each partition of the validation data
#     # this findes the np.mean of those accuracies to get the overall accuracy for the weights
#     acc.append(np.mean(dd.compute(*[evaluate(dvl,weights[j]) for dvl in data])))
# #The weights that produced the best accuracy 
#     weights_best = weights[np.argmax(acc)]
# acc


In [25]:
import g6 #our package we defined earlier, has weight file saving functions and the cnn variable

#The heart and soul
#This ties it all together to distribute the training of the cnn. On n workers we train data/n on each worker. data/n is split into chunks manageable on the ram. A worker loads in a chunk of its data/n and trains the cnn on it for 10 epochs. Then it saves the weights to a file. When it gets the net chunk it loads the last weights and trains on top of them again. After it has repeated this for all of its chunks, we have a set of weights that has been trained on data/n. This happens for all n workers, then we have n sets of weights.

#Then we do the distributed evaluation. First we evaluate the accuracy of w1 then w2 then wn. This happens in sequence. The evaluation of each one is distributed in paralell. Each worker has its data_eval/n saved locally in chunks. It loads one chunk at a time and evaluated the cnn accuracy with the weights being tested. Each worker returnes a collection of accuracies for each chunk evaluated. Finally all chunk accuracies are returned to this jupyter machine. Here we find the mean to know the true accuracy of weights w1 over all of the data. This is appended to a list. This is repeated for w2, w3, wn. The weights that returned the best accuracy is kept and saved to each worker for another training. 

#This repeats for nitter itterations.
#images per chunk allows us to control how much the workers have to handle. I found chunking by bytes to be too inaccurate.
#frac_tr is the ratio of training to validation data. I suspect this should be kept low. Our goal is good performance analysis not a good cnn. With less training data it takes longer to train which amplifies performance gaps between systems. If both systems get 0.99 accuracy in 0.2 +- 0.02 seconds its difficult to say which is better. But if one is an inefficiant model it will show as each inefficiency adds up in a longer run. Also with a large validation set we get more accurate accuracy readings. This means our performance analysis for time to 0.95 accuracy is more reliable.    
data = getData()
def cnn_dist(data, frac_tr = 0.2, images_per_chunk=50, nitter=5):
    print('Splitting up the data into training and validation sets')
    m_t = int(len(data)*frac_tr)
    data_tr = data[:m_t].rechunk(chunks=(images_per_chunk/2,2,198,198,3)).to_delayed().ravel()
    data_val = data[m_t:].rechunk(chunks=(images_per_chunk/2,2,198,198,3)).to_delayed().ravel()
    
    del data, frac_tr, m_t, images_per_chunk
    
    #initialising the weights stored locally on the workers
    client.run(g6.weights_init)

    for i in range(nitter):
        print('Training Itteration:',i)
        #train on the distributed training dataframe, loading and saving the weights from a file
        dd.compute(*[train(dtr) for dtr in data_tr])

        print('getting trained weights:',i)
        #the weights have been trained. This brings them from the workers to this machine for validation
        weights = list(client.run(g6.weights_get).values())
        acc = []
        for j in range(len(weights)):
            print('evaluation at itteration:',i,'of weights',j)
            #distrbuted evaluation of weights, returns accuracies of each partition of the validation data
            # this findes the np.mean of those accuracies to get the overall accuracy for the weights
            acc.append(np.mean(dd.compute(*[evaluate(dvl,weights[j]) for dvl in data_val])))
        #The weights that produced the best accuracy 
        weights_best = weights[np.argmax(acc)]

        del weights
        best_acc = np.max(acc)
        print('The best accuracy amongst the weights found is,',best_acc,
              '\n Setting the corresponding best weights for itteration:',i)
        client.run(g6.weights_set_best, weights_best)
        #The best weights found have been set, ready for the next training itteration

    return np.max(acc), weights_best

acc_best, weights_best = cnn_dist(data)

Splitting up the data into training and validation sets
Training Itteration: 0
getting trained weights: 0
evaluation at itteration: 0 of weights 0
evaluation at itteration: 0 of weights 1
evaluation at itteration: 0 of weights 2
evaluation at itteration: 0 of weights 3
evaluation at itteration: 0 of weights 4
evaluation at itteration: 0 of weights 5
The best accuracy amongst the weights found is, 0.9243749929592013 
 Setting the corresponding best weights for itteration: 0
Training Itteration: 1
getting trained weights: 1
evaluation at itteration: 1 of weights 0
evaluation at itteration: 1 of weights 1
evaluation at itteration: 1 of weights 2
evaluation at itteration: 1 of weights 3
evaluation at itteration: 1 of weights 4
evaluation at itteration: 1 of weights 5
The best accuracy amongst the weights found is, 0.9293750002980232 
 Setting the corresponding best weights for itteration: 1
Training Itteration: 2
getting trained weights: 2
evaluation at itteration: 2 of weights 0
evaluatio

In [23]:
print("The best accuracy found from the distributed cnn training is, ",acc_best)

The best accuracy found from the distributed cnn training is,  0.9164999961853028
